## LAB: La maldición de la dimensionalidad

Para entender el efecto de la cantidad de dimensiones en la capacidad predictiva del modelo vamos a trabajar en un clasificador de noticias de dos diarios argentinos para intentar distinguir entre uno y otro a partir del vocabulario que utilizan. <br />

Para esto vamos a implementar un modelo de tipo Naïve Bayes con vectorización de tipo TF-IDF.

In [ ]:
import pandas as pd
df_clarin = pd.read_csv('clarin.csv')
df_clarin['class'] = 0

In [ ]:
df_p12 = pd.read_csv('pagina12.csv')
df_p12['class'] = 1

In [ ]:
df = pd.concat([df_clarin,df_p12])
df.sample(5)

### 1. Limpieza

#### 1.1 Faltantes

A partir del dataset observamos que los campos que probablemente contengan el vocabulario relevante son "cuerpo", "título" y "resumen".
Sacar del análisis los registros que no tienen cuerpo o título disponible y completar los resúmenes faltantes con una campo en blanco


In [ ]:
df = df[df['cuerpo'].notnull()].copy()

In [ ]:
df = df[df['titulo'].notnull()].copy()

In [ ]:
df['resumen'].fillna('',inplace=True)

#### 1.2 Suplementos relevantes

Para mejorar la clasificación es conveniente retirar las secciones donde los dos diarios utilizan un vocabulario similar y muy específico del dominio como, por ejemplo, las relacionadas a deportes.

In [ ]:
# Filtramos las noticias relacionadas a deportes
df = df[df['suplemento'].str.lower().str.contains('deportes')==False].copy()
df['suplemento'].value_counts()

#### 1.3 Corpus

Construir la columna sobre la cual vamos a predecir concatenando el título, resumen y cuerpo de las distintas noticias.

In [ ]:
df['full'] = df['cuerpo'].astype('str') + df['titulo'] + df['resumen']

#### 1.4 Normalización del texto

Primero es conveniente pasar todo el corpus a letras minúsculas.

Para mejorar el clasificador es importante sacar todos los acentos que pueden generar diferencias artificiales entre palabras según estén o no puestos. Hint: Utilizar el módulo unidecode

Una vez retirados los acentos, quitar todos los signos de puntuación para dejar únicamente palabras. Hint: utilizar una expresión regular, por ejemplo, r'([^\s\w]|_)+' para reemplazar todo lo que no sean palabras.

In [ ]:
df['full'] = df['full'].str.lower()

In [ ]:
from unidecode import unidecode
df['full'] = df['full'].apply(unidecode)

In [ ]:
df['full'].sample(3)

In [ ]:
import re
df['full'] = df['full'].apply(lambda x: re.sub(r'([^\s\w]|_)+', '', x))

In [ ]:
df['full'].sample(3)

## 2. Modelo

Vectorizar el corpus resultante con TF-IDF y aplicar un modelo Naive Bayes con un split simple entre train y test. 
<br />
¿Cuál es la dimensión de la matriz de features? ¿Cuál es el accuracy obtenido? <br />
Dibujar la matriz de confusión.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df['full'], df['class'], test_size=0.33, random_state=42) 

In [ ]:
# Sólo para chequear el tamaño de la matriz de features
from sklearn.feature_extraction.text import TfidfVectorizer
m1 = TfidfVectorizer()
X_train_vec = m1.fit_transform(X_train)
X_train_vec.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

model = make_pipeline(TfidfVectorizer(), MultinomialNB())

In [ ]:
model.fit(X_train, y_train)
labels = model.predict(X_test)
labels

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, labels)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns;
import matplotlib.pyplot as plt
%matplotlib inline
mat = confusion_matrix(y_test, labels)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=['Clarin','Pagina'], yticklabels=['Clarin','Pagina'])
plt.xlabel('true label')
plt.ylabel('predicted label');

## 3. Reducción de la dimensionalidad

¿Cuál es la relación entre la cantidad de features y la cantidad de casos de entrenamiento? De acuerdo a la maldición de la dimensionalidad esta relación es un problema.<br/>
Una de las técnicas más comunes para reducir dimensiones es utilizar "stopwords", una lista con las palabras de cada idioma que no tienen un peso semántico importante. <br />
Entrenen nuevamente el modelo quitando las stopwords del español que se encuentran en el archivo "stopwords.csv". <br />
¿Qué pasa con la cantidad de dimensiones de la matriz de features? ¿Y con el accuracy?

In [ ]:
stopwords = pd.read_csv('stopwords.csv',header=None)[0]

In [ ]:
from unidecode import unidecode
stopwords = stopwords.apply(unidecode) 

In [ ]:
# Es importante que las stopwords se encuentren en una lista
stopwords = list(stopwords)

In [ ]:
# Sólo para chequear el tamaño de la matriz de features
m1 = TfidfVectorizer(stop_words = list(stopwords))
X_train_vec = m1.fit_transform(X_train)
X_train_vec.shape

In [ ]:
model = make_pipeline(TfidfVectorizer(stop_words = list(stopwords)), MultinomialNB())

In [ ]:
model.fit(X_train, y_train)
labels = model.predict(X_test)

In [ ]:
accuracy_score(y_test, labels)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns;
import matplotlib.pyplot as plt
%matplotlib inline
mat = confusion_matrix(y_test, labels)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=['Clarin','Pagina'], yticklabels=['Clarin','Pagina'])
plt.xlabel('true label')
plt.ylabel('predicted label');

## 4. Avanzando en la reducción de dimensiones

El modelo mejora cuando quitamos las stopwords pero todavía tenemos una cantidad de dimensiones demasiado alta para 
la cantidad de datos que tenemos.

Pensemos lo siguiente ¿Qué valor tienen las palabras que aparecen una o dos veces en todo el corpus a la hora de discriminar? Para remover del análisis las palabras que ocurren menos de determinada cantidad de veces, la clase TfidfVectorizer tiene un parámetro min_df. 

Prueben setear el parámetro min_df en 6 y volver a correr el modelo.
<br />
<strong>
¿A cuánto se reduce la dimensión de la matriz de features?
<br />
¿Cuánto mejora la performance del algoritmo?
<strong />

In [ ]:
model = make_pipeline(TfidfVectorizer(stop_words = list(stopwords), min_df=6), MultinomialNB())

In [ ]:
model.fit(X_train, y_train)
labels = model.predict(X_test)

In [ ]:
model_vec2 = TfidfVectorizer(stop_words = list(stopwords), min_df=10)




In [ ]:
X_train_vec = model_vec2.fit_transform(X_train)

In [ ]:
X_train_vec.shape

In [ ]:
accuracy_score(y_test, labels)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns;
import matplotlib.pyplot as plt
%matplotlib inline
mat = confusion_matrix(y_test, labels)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=['Clarin','Pagina'], yticklabels=['Clarin','Pagina'])
plt.xlabel('true label')
plt.ylabel('predicted label');

In [ ]:
pred = model.predict(['La candidata a senadora de Unidad Ciudadana dijo que la decisión de la Sala II de la Cámara Federal porteña ordenar la detención de Julio De Vido es parte de la campaña impulsada por el macrismo. En una entrevista con Telefe cuestionó también al Gobierno por la defensa de la Gendarmería frente a la desaparición de Santiago Maldonado.'])
print('Clarin' if pred[0] == 0 else 'Página 12')

In [ ]:
pred = model.predict(['La Cámara exigió la detención de De Vido por su influencia para destruir pruebas '])
print('Clarin' if pred[0] == 0 else 'Página 12')